In [ ]:
using Polly;
using System;
using System.Net;
using System.Threading.Tasks;
using RestSharp;

public class RetryPolicyExample
{
    private static readonly AsyncRetryPolicy<RestResponse> RetryPolicy = Policy
        .HandleResult<RestResponse>(resp => 
            resp.StatusCode == HttpStatusCode.TooManyRequests || 
            (int)resp.StatusCode >= 500
        )
        .WaitAndRetryAsync(
            retryCount: 4, 
            sleepDurationProvider: retryAttempt => TimeSpan.FromSeconds(5 + retryAttempt),
            onRetry: (response, timespan, retryCount, context) =>
            {
                Console.WriteLine($"Retry {retryCount} after {timespan.TotalSeconds} seconds due to: {response.Result.StatusCode}");
            }
        );

    public static async Task RunRetryPolicyAsync()
    {
        var client = new RestClient("https://api.example.com/resource");
        var request = new RestRequest();

        var response = await RetryPolicy.ExecuteAsync(async () =>
        {
            return await client.ExecuteAsync(request);
        });

        Console.WriteLine($"Final response status code: {response.StatusCode}");
    }
}

await RetryPolicyExample.RunRetryPolicyAsync();
